In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/arenda-torgovyh-ploschadey?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

269


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 7/7 [00:02<00:00,  2.77it/s]

244


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Аренда торговых площадей'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 244/244 [01:00<00:00,  4.00it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Площадь,Этаж / этажность,Высота потолков,...,Интернет,Мебель,Телефон,Этажность,Материал стен,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/arenda-pomescheniy/ul-a...,"Аренда помещения на ул. Алексеевская, д. 10/16...",230 000 руб./мес.,Торговое помещение ( помещение свободного назн...,2809634,Нижегородский,"ул. Алексеевская, д. 10/16",115 м²,2 / 6,"3,5 м",...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда торговых площадей,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/arenda-pomescheniy/ul-b...,"Аренда помещения на ул. Бекетова, д. 13аСоветс...",17 500 руб./мес.,Сдается от собственника торговое место под выс...,2834800,Советский,"ул. Бекетова, д. 13аТЦ Бекетов",5 м²,1 / 3,4 м,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда торговых площадей,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/arenda-pomescheniy/ul-r...,"Аренда помещения на ул. Республиканская, д. 14...",50 000 руб./мес.,Сдам нежилое помещение в Советском районе на у...,2862942,Советский,"ул. Республиканская, д. 14","74,4 м²",1 / 14,Уточнить,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда торговых площадей,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/arenda-pomescheniy/ul-b...,"Аренда помещения на ул. Бориса Корнилова, д. 6...",87 850 руб./мес.,"Помещение подвальное\r\nул Бориса Корнилова,\r...",2860317,Советский,"ул. Бориса Корнилова, д. 6 к1",251 м²,-1 / 5,"2,8 м",...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда торговых площадей,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/arenda-pomescheniy/ul-k...,"Аренда помещения на ул. Культуры, д. 10Сормовс...",100 000 руб./мес.,Предлагается в аренду часть торгового помещени...,2836884,Сормовский,"ул. Культуры, д. 10",56 м²,1 / 9,Уточнить,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда торговых площадей,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [7]:
df.to_excel('trade-rent.xlsx', index=False)